# Setup Requirements

## Import Libraries

In [ ]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

## Specify Paths

In [ ]:
m_dataset_path = 'model/data/m_data.csv'
m_model_save_path = 'model/m_model.keras'
m_tflite_save_path = 'model/m_model.tflite'

## Setup Attributes

In [ ]:
# Data Attributes
M_NUM_CLASSES = 3
M_SEQ = 21
REFERENCE_LANDMARKS_NUM = 21
DIMENSION = 2

# Model Attributes
RANDOM_SEED = 42
TRAIN_SIZE = 0.8
TRAIN_SPLIT_SIZE = 0.75
EPOCHS = 150
BATCH_SIZE = 8
EARLY_STOPPING_PATIENCE = 8

# Preprocess Data

## Load Data

In [ ]:
X_m_dataset = np.loadtxt(m_dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (M_SEQ * (REFERENCE_LANDMARKS_NUM * DIMENSION)) + 1)))

In [ ]:
y_m_dataset = np.loadtxt(m_dataset_path, delimiter=',', dtype='int32', usecols=(0))

## Split Train/Test

In [ ]:
# 60/20/20 Train/Validation/Test Split 
X_m_train, X_m_test, y_m_train, y_m_test = train_test_split(X_m_dataset, y_m_dataset, train_size=TRAIN_SIZE, random_state=RANDOM_SEED)
X_m_train, X_m_val, y_m_train, y_m_val = train_test_split(X_m_train, y_m_train, train_size=TRAIN_SPLIT_SIZE, random_state=RANDOM_SEED)

# Build Model

## Construct Model

In [ ]:
# Bi-LSTM for time series
m_model = tf.keras.models.Sequential([
    tf.keras.layers.Input((M_SEQ * (REFERENCE_LANDMARKS_NUM * DIMENSION),)),
    tf.keras.layers.Reshape((M_SEQ, (REFERENCE_LANDMARKS_NUM * DIMENSION)), input_shape=(M_SEQ * (REFERENCE_LANDMARKS_NUM * DIMENSION), )),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.2, input_shape=[(M_SEQ, (REFERENCE_LANDMARKS_NUM * DIMENSION))], activation='tanh')),
    #tf.keras.layers.LSTM(64, return_sequences=False, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(M_NUM_CLASSES, activation='softmax')
])

## Compile Model

In [ ]:
# Model compilation
m_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## Setup Callbacks

In [ ]:
# Model checkpoint callback
m_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    m_model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, verbose=1)

## Summary

In [ ]:
m_model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

# Train Model

In [ ]:
# Cross validate with validate set
m_model.fit(
    X_m_train,
    y_m_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_m_val, y_m_val),
    callbacks=[m_cp_callback, es_callback]
)

# Evaluate Model

## Plot Confusion Matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_m_test, y_pred))

Y_pred = m_model.predict(X_m_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_m_test, y_pred)

## Accuracy and Loss

In [ ]:
val_loss, val_acc = m_model.evaluate(X_m_test, y_m_test, batch_size=BATCH_SIZE)

## Predict Unseen Data

In [ ]:
predict_result = m_model.predict(np.array([X_m_test[0]]))
print(predict_result)
print(np.squeeze(predict_result))
print(np.sum((predict_result)))
print(np.argmax(np.squeeze(predict_result)))

# Convert TFLite

## Save Model

In [ ]:
m_model.save(m_model_save_path)

## Load Model

In [ ]:
m_model = tf.keras.models.load_model(m_model_save_path)

## Quantization & Save

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(m_model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]
tflite_quantized_model = converter.convert()

open(m_tflite_save_path, 'wb').write(tflite_quantized_model)

# Inference test

## Setup Interpreter Interface

In [ ]:
interpreter = tf.lite.Interpreter(model_path=m_tflite_save_path)
interpreter.allocate_tensors()

## Get Tensor Details

In [ ]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## Implement Inference

### Setup Input Tensor

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_m_test[0]]))

### Invoke Interpreter

In [ ]:
%%time
interpreter.invoke()

### Get Tensor Output

In [ ]:
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))